In [1]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, LineString
import os
import json
import gzip
import time
import calendar
from datetime import datetime
import numpy as np
import pandas as pd
from datetime import date
import warnings
warnings.filterwarnings('ignore')
SETS = './sets/' # папка с файлами с наборами
LONG = 0.0159 # долгота расстояние 1 км от точки на восток или на запад
LAT = 0.009 # широта расстояние 1 км от точки на север или на юг

In [2]:
def default_serializer(obj): # для правильного преобразования даты в ISO формат
    if isinstance(obj, (date)):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')
    

def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')
        

# Функция для преобразования данных в геодатафрейм
def create_geodataframe(row, geo_column):
    try:
        geo_data = row[geo_column]

        # Проверяем, является ли geo_data NaN
        if pd.isna(geo_data):
            return gpd.GeoDataFrame(geometry=[None])  # Возвращаем None, если geo_data NaN

        geometry_type = geo_data['type']
        
        if geometry_type == 'Point':
            coordinates = geo_data['coordinates']
            geometry = Point(coordinates)
        elif geometry_type == 'Polygon':
            coordinates = geo_data['coordinates'][0]  # Предполагаем, что координаты переданы в виде (объект)
            geometry = Polygon(coordinates)
        elif geometry_type == 'MultiPolygon':
            geometry = MultiPolygon([Polygon(p[0]) for p in geo_data['coordinates']])
        elif geometry_type == 'LineString':
            coordinates = geo_data['coordinates']
            geometry = LineString(coordinates)
        else:
            # Если тип геометрии не распознан, возвращаем None
            geometry = None
    
    except Exception as e:
        print(f"Ошибка: {e}. Значение: {row[geo_column]}")  # Выводим ошибку и значение, вызвавшее ошибку
        geometry = None  # Возвращаем None в случае ошибки

    return gpd.GeoDataFrame(geometry=[geometry])

# Координаты

Для определения местоположения объекта в пространстве используют координаты.

**Координаты** - это числовые значения, используемые для определения положения точки или объекта в пространстве или на плоскости. Координаты представляют собой систему измерений, которая позволяет точно описать местоположение объекта относительно некоторой точки или оси. 
Измерения можно осуществлять разными способами, например, метоположение ручки на столе можно задать с помощью расстояний от его краев, причем от разных, либо от верхнего края и левого или нижнего и правого и т.д. Но, в тамо слуае, другому человеку считавшего эти измерения, например с листка бумаги, будет не ясно какие измерения производить для точного определения местоположения ручки. Для стандартизации измерения, существуе множество **систем координат**.

**Систе́ма координа́т** — комплекс определений, реализующий метод координат, то есть способ определять положение и перемещение точки или тела с помощью чисел или других символов. Совокупность чисел, определяющих положение конкретной точки, называется координатами этой точки.

Наиболее используемая система координат — прямоугольная система координат (также известная как декартова система координат). 

В данной системе используется горизонтальная ось — ось X, вертикальная — ось Y.

Значения на оси X называют абсциссами, а значения на оси Y — ординатами.

Для определения местоположения объекта на плоскости земли, так же  используют оси X и Y.

Значение на оси X называют **долготой**, по оси Y — **широтой**.

Линии опоясывающие весь земной шар, проведенные через все долготы, называт **мередианами**

Параллельные линии опоясывающие землю по широтам, называют **параллелями**.

Параллели и мередианы имеют форму окружности. Любая окружность имеет 360 градусов. Поэтому точки на меридианах и параллелях выражуются через градусы, минуты и секунды.
Но, в современных сервисах и публичных картах географические координыты переведены для удобства в десятичное представление, где в дробной части нет минут и секунд.

Например, координаты долготы $37^\circ 45' 30''$ (37 градусов, 45 минут и 30 секунд), переводят в десятичное представление следующим образом:

```
37 + (45 / 60) + (30 / 3600) = 37.7583
```

Большинство публичных сервисов и карт отображают десятичную форму записи координат.

## Порядки представляния координат

Основная проблема с координатами заключается в разном порядке предсталвения широты и долготы. 

Существует два вида порядка представления значений широты и долготы:

1. **longlat** - прямое (классическое) представление, где первое значение X (долгота), а второе Y (широта)
2. **latlong** - обратное предствление, где  первое значение Y (широта), второе значение X (долгота)

Многие сервисы и публичные карты (Google Maps, Bing Maps, OpenStreet Maps) придерживаются второго (обратного) представления, где сначала указывают ординату (широту), далее абсциссу (долготу). 

Пример координат (точка в Москве) в обратном представлении:

```
55.814547, 37.723652
```

Долгое время Яндекс Карты придерживались "правильного" прямого представления значений координат:

```
37.723652, 55.814547
```

Остались следы ["железных аргументов"](https://yandex.ru/blog/mapsapi/8089#coord-3) в правильности такого подхода на их сайте. Но, в конечном итоге, они перешли на более распрастранненое обратное представления, 

Ресурс data.mos.ru появился 11 лет назад, в то время Яндекс Карты использвали прямое представление значений координат и скорее всего из-за уже имеющихся на тот момент данных не перешли вслед за Яндекс Картами к обратному виду.

## Значения широты и долготы

Разссмотрим подробнее значения широты и долготы, их особенности, форматы.

### Меридиан Гринвича

Нулевой мередиан или нулевой долготный градус начинатяется от Гринвича (район в Лондоне), от которого начинается не только нулевой долготный круг, но и время GMT (GMT - Greenwich Mean Time) 


### Долгота

На лево на восток от Гринфича значение градусов может быть от 0 до 180 градусов. 

Пример классической записи долготы:

$37^\circ 45' 30''$ В.Д. (восточной долготы)

Для сокращения записи используют десятичную со знаком плюс, если долгота восточная и минус если западная

Пример, десятичная запись долготы на восток от Гринвича:

37.7583 

Мередиан с противоположной стороны от Гринвича (180 градусов) называют: Меридиан Антарктики или Меридиан Международной даты. Там проходит линия перемены дат, разделяющая два календарных дня.

### Экватор

Нулевую параллель называют экватором, она делит земной шар на два полушария южное и севрное. Линии параллельные экватору называют параллелями, а значение параллелей называют широтой.

#### Широта

Паралелли севернее экватора называют - **северная широта**. Она измеряется в градусах от 0 до 90 на северн от экватора. Северная широта обозначается положительными значениями.

**Южная широта**: измеряется в градусах от 0 до 90 южнее от экватора. Южная широта обозначается отрицательными значениями.

Выделяются следующие широты (параллели):

- Полюс (90 градусов северной и южной широты) - точки, где ось вращения Земли пересекает поверхность.
- Северный и Южный Полярные круги (66,5 градусов северной и южной широты) - где Солнце может быть видно в течение 24 часов в летний сезон.
- Тропики Рака и Козерога (23,5 градусов северной и южной широты) - места, где Солнце может находиться в зените во время солнцестояний.
- Субтропики 23,5 до 40 градусов
- Параллель 60 градусов северной широты - широта, к которой обычно относятся многие столицы и крупные города.
- Параллель 45 градусов северной широты - проходит через множество известных винодельческих регионов.


## Системы навигации

- GPS (Global Positioning System) - это глобальная система спутниковой навигации
- ГЛОНАСС (Глобальная Навигационная Спутниковая Система) - российская аналогичная система спутниковой навигации.
- Galileo - европейская система спутниковой навигации.
- BeiDou - китайская система спутниковой навигации.
- IRNSS (Indian Regional Navigation Satellite System) - индийская региональная система спутниковой навигации.


## Структуры данных

В датасете данные с координатами хранятся:

- в словаре - как правило один адрес на объект, либо несколько адресов записаны с типом MuliPolygon
- в списке словарей - как правило это несколько адресов у одного объекта, предоставлены без особого типа

### Dict

#### Геометрический тип объекта

Многие сервисы и карты могут возвращать значения координат объекта с одним или несколькими геометрическими типами. Как правило сервисы предпочитают конкретный геометрический тип, который может отличаться от типов других служб. Но, все они стараются придерживаться международных стандартов по типам, одним из которых является RFC 7946 (GEOJSON.org)

В соответствии со стандартом GOEJSON существует несколько геометрических типов:

- Point
- LineString
- Polygon
- MultiPoint
- MultiLineString
- MultiPolygon

В соответсвии со стандартными типами, в дипломной работе используются свпецифичные для конкретной службы геометрии, а так же с учетом типов не предусмотренных стандартом, но имеющихся в датасетах есть слетдущие типы:

##### 1. Точка (Point):

```
{"coordinates":[37.505049,55.523081],"type":"Point"}
```

или

```
{"type": "Point", "coordinates": [37.775102, 55.806463]}
```

##### 2. Несколько точек (MultyPoint):

```
{"coordinates":[[37.58037861,55.825088698],[37.580234128,55.824253023],[37.579996
429,55.824365685],[37.58015831,55.82440844],[37.580110955,55.824520978],[37.579997166,55.824724637],[37.579883299,55.824890793],[37.581054021,55.824932887],[37.580624454,55.8243117
],[37.580910526,55.824579387],[37.580834711,55.824734804],[37.580720825,55.824890246],[37.580606916,55.825034973],[37.580558908,55.82483142],[37.580520562,55.824697508]],"type":"MultiPoint"}
```

##### 3. Рамка (boundingbox):


```
{'type': 'bbox', 'coordinates': [[37.7505766 ,55.6519879], [37.7501636, 55.6522925]]}
```
Первая координата указывают на нижнюю левую точку прямоукольника, вторая на верхний правый


##### 4. Ломанная линия (MultiLineString):

```
{"coordinates":[[[37.636096932,55.872757589],[37.635454436,55.872703212],[37.6355
08176,55.872504232],[37.635426154,55.87239721],[37.635479989,55.872225772],[37.636496897,55.871949219],[37.636534859,55.871909392],[37.636565665,55.871912656],[37.636618858,55.8717
73],[37.636961989,55.871799406],[37.636965382,55.87191735],[37.636966046,55.871936853],[37.637545107,55.871988443],[37.637528583,55.872031321],[37.636779133,55.871962677],[37.63675
6362,55.871974816],[37.63655118,55.871950683],[37.636543324,55.871971309],[37.635829634,55.872250687],[37.635687305,55.872686137],[37.635521575,55.872670514],[37.635560958,55.87248
698],[37.635501808,55.872382798],[37.635551735,55.872248253],[37.63655118,55.871950683],[37.636565665,55.871912656],[37.636763557,55.871933621]],[[37.636285173,55.872742567],[37.63
5687305,55.872686137],[37.636119945,55.872726918]],[[37.636215263,55.872847282],[37.636081819,55.872836715],[37.636110022,55.872726856],[37.636253004,55.872740091],[37.636220028,55
.872847277]],[[37.636767681,55.871944891],[37.636756362,55.871974816],[37.636693534,55.87200831],[37.636608799,55.872300385],[37.636453179,55.87275862],[37.63629113,55.872742728]]]
,"type":"MultiLineString"}
```

##### 5. Мултиполигон (MultiPolygon):

```
{'coordinates': [[[[37.710083127, 55.825085846], [37.709884644, 55.825088859], [37.70991683, 55.825557425], [37.710107267, 55.825552906], [37.710083127, 55.825085846]]], [[[37.710971466, 55.824666254], [37.710390321, 55.824675741], [37.710401249, 55.824826407], [37.710980407, 55.824816363], [37.710971466, 55.824666254]]], [[[37.710757032, 55.82506651], [37.710557953, 55.825070528], [37.710585371, 55.825566047], [37.710780874, 55.825561359], [37.710757032, 55.82506651]]], [[[37.707049481, 55.825154524], [37.707242202, 55.825149502], [37.707241209, 55.825030085], [37.70704054, 55.82503176], [37.707049481, 55.825154524]]], [[[37.710930347, 55.825726055], [37.711380243, 55.825718823], [37.711373091, 55.825557312], [37.711369514, 55.825539635], [37.710924625, 55.825550081], [37.710924625, 55.825570973], [37.710930347, 55.825726055]]], [[[37.583026114, 55.832647996], [37.582666499, 55.832611174], [37.582629743, 55.832706019], [37.582630736, 55.832758463], [37.58344732, 55.83284717], [37.583486063, 55.832741167], [37.583481096, 55.83270044], [37.583129428, 55.832659155], [37.583156251, 55.832582163], [37.584034426, 55.832677566], [37.584082109, 55.83255092], [37.584084096, 55.832518561], [37.583018166, 55.832398051], [37.582968496, 55.832531393], [37.582976443, 55.832562078], [37.583054923, 55.832573236], [37.583026114, 55.832647996]]], [[[37.709097862, 55.825586152], [37.70911789, 55.825928458], [37.70928669, 55.825930065], [37.709272385, 55.825774179], [37.70981884, 55.825766144], [37.709795952, 55.825566867], [37.709097862, 55.825586152]]]], 'type': 'MultiPolygon'}
```

##### 6. Геометрическая коллекция (GeometryCollection) 

Может включать все вышеперечисленные типы в одном наборе:

```
{"geometries":[{"coordinates":[[[37.506825543,55.685344001],[37.506928632,55.6855
10346],[37.507208483,55.686142464],[37.507031932,55.686841148],[37.506516704,55.687365182],[37.505898373,55.687739503],[37.505309432,55.687781118],[37.504926623,55.68783104],[37.50
4823565,55.687897584],[37.504722591,55.687800154],[37.504604802,55.687804911],[37.504318751,55.687890474],[37.504209387,55.687999794],[37.504184152,55.688056829],[37.504200993,55.6
88180404],[37.504091615,55.688170902],[37.503940157,55.688047332],[37.503822355,55.687928514],[37.503704562,55.687895247],[37.503494223,55.687895253],[37.503258638,55.68782872],[37
.503115606,55.687814465],[37.503073531,55.687719408],[37.503048282,55.687610092],[37.502821107,55.687467511],[37.502661247,55.687410481],[37.50239202,55.687458015],[37.502408852,55
.687543567],[37.502248995,55.687534064],[37.502139623,55.687591101],[37.502072321,55.687690912],[37.501575927,55.687771719],[37.501289868,55.687795487],[37.501146839,55.687843017],
[37.500852367,55.687914313],[37.500339139,55.687852528],[37.500154041,55.687852529],[37.499960529,55.687952339],[37.4996324,55.687961845],[37.499405236,55.687838269],[37.49917807,5
5.687838268],[37.498892003,55.688061651],[37.498908826,55.688213744],[37.498833103,55.688237507],[37.49865642,55.688123437],[37.49834512,55.688061645],[37.498185265,55.688009361],[
37.497705702,55.687819238],[37.497419651,55.687662388],[37.497285029,55.687757443],[37.498084299,55.688080653],[37.498479732,55.688208986],[37.498799447,55.688313553],[37.498959305
,55.688337319],[37.499068685,55.688208992],[37.499060273,55.688147204],[37.499043446,55.68812344],[37.499211721,55.68799036],[37.499295858,55.687933326],[37.499531437,55.688042644]
,[37.500027838,55.688037891],[37.500179282,55.68801888],[37.500229764,55.687966598],[37.50057472,55.687990361],[37.500978573,55.688018875],[37.50142449,55.687933319],[37.501466556,
55.687900048],[37.501786274,55.687966584],[37.502249013,55.687833496],[37.502291075,55.687733684],[37.502274242,55.687638627],[37.502392032,55.687652883],[37.502543471,55.687600598
],[37.502585532,55.68750554],[37.502947327,55.687705153],[37.50297258,55.687861997],[37.503208163,55.687904767],[37.503325954,55.687919022],[37.503628849,55.68799506],[37.503704572
,55.68799981],[37.503898105,55.688204178],[37.504100037,55.688256453],[37.504251488,55.688303977],[37.504369262,55.688156633],[37.504335599,55.688075836],[37.504461791,55.687971268
],[37.504621642,55.687914227],[37.504840398,55.687942736],[37.505160102,55.687857171],[37.505496642,55.687842897],[37.505908896,55.68778109],[37.506169691,55.687624232],[37.5067080
97,55.687291502],[37.506952033,55.686992057],[37.507170729,55.686716377],[37.507221161,55.686459718],[37.507263166,55.686131766],[37.507178997,55.685927397],[37.507094829,55.685727
781],[37.506918095,55.685395089],[37.506876016,55.685319046],[37.506825542,55.68533806]]],"type":"MultiLineString"},{"coordinates":[[[37.501735789,55.68787628],[37.5020555,55.68780
4982],[37.502005015,55.687738442],[37.501651651,55.687819247],[37.501735789,55.68787628]]],"type":"Polygon"}],"type":"GeometryCollection"}
```

### list

##### 7. Список

Содержит несколько адресов, разных геометрических типов:

```
{'type': 'bbox', 'coordinates': [[37.5674465, 55.855477], [37.5685451, 55.8556698]]}, {'type': 'bbox', 'coordinates': [[37.5671809, 55.8561421], [37.5672809, 55.8562421]]}, {'type': 'bbox', 'coordinates': [[37.5689295, 55.8558889], [37.5692307, 55.8561655]]}, {'type': 'bbox', 'coordinates': [[37.5674772, 55.8564831], [37.5677811, 55.8567601]]}, {'type': 'bbox', 'coordinates': [[37.5690709, 55.8562977], [37.5693604, 55.85658]]}]
```

## Координаты в киломметры

### Широта

Возьмем "круглую" точку (это где-то в печатниках)

```
55.7 37.7
```

Прибавим одну сотую к широте, на севере в 1.11 км появилась метка

```
55.71 37.7
```

Отнимим сотую, на юге точка в 1.11 км

```
55.69 37.7
```

следовательно 0.01 по широте это 1.11 км. Следовательно по широте **0.009** - это километр

### Долгота

Прибавим одну сотую к долготе, на востоке появилась точка в 629 метрах

```
55.7 37.71
```

Уменьшим сотую, на западе точка в 629 метрах

```
55.7 37.69
```

Следовательно 0.01 по долготе это 629 метров. Следовательно по долготе **0.0159** - это киллометр

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [4]:
ecology = load_dataset('ecology')
ecology['start'] = pd.to_datetime(ecology['start'])
ecology['end'] = pd.to_datetime(ecology['end'])
exams = load_dataset('exams')
exams['start'] = pd.to_datetime(exams['start'])
exams['end'] = pd.to_datetime(exams['end'])

In [5]:
#%%time
#ecology['air'] = ecology.apply(lambda row: create_geodataframe(row, 'air') if not pd.isna(row['air']) else None, axis=1)
ecology[ecology['digging'].notna()]

,start,end,air,mean,noise,dirt,digging,capital,house_lines,clinics,roads,parks,transports,city_places,live_area,streets,trashes,factories,markets,autostations,gasstations,bad_gasstations,stroi
3001,2021-02-22,2024-09-30,NaN,NaN,NaN,NaN,"{'coordinates': [[37.58037861, 55.825088698], [37.580234128, 55.824253023], [37.579996429, 55.824365685], [37.58015831, 55.82440844], [37.580110955, 55.824520978], [37.579997166, 55.824724637], [37.579883299, 55.824890793], [37.581054021, 55.824932887], [37.580624454, 55.8243117], [37.580910526, 55.824579387], [37.580834711, 55.824734804], [37.580720825, 55.824890246], [37.580606916, 55.825034973], [37.580558908, 55.82483142], [37.580520562, 55.824697508]], 'type': 'MultiPoint'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3002,2021-08-23,2024-08-23,NaN,NaN,NaN,NaN,"{'coordinates': [[[37.503629953, 55.670195783], [37.506739318, 55.670206366], [37.506758129, 55.669349144], [37.503610916, 55.669359994], [37.503629953, 55.670195783]]], 'type': 'Polygon'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3003,2021-09-09,2024-09-02,NaN,NaN,NaN,NaN,"{'coordinates': [[[37.915712894, 55.709283775], [37.915802417, 55.709448219], [37.915942536, 55.709462478], [37.915997848, 55.709309598], [37.915712894, 55.709283775]]], 'type': 'Polygon'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3004,2021-10-07,2024-12-31,NaN,NaN,NaN,NaN,"{'coordinates': [[[37.757165076, 55.807141332], [37.749655174, 55.805013798], [37.748938865, 55.80605461], [37.756296225, 55.8081075], [37.757165076, 55.807141332]]], 'type': 'Polygon'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3005,2021-11-20,2024-08-30,NaN,NaN,NaN,NaN,"{'coordinates': [[[37.757317925, 55.708772863], [37.758517205, 55.708606936], [37.758247238, 55.707959233], [37.757507798, 55.70806526], [37.757358784, 55.707770903], [37.756974752, 55.70781189], [37.757317925, 55.708772863]]], 'type': 'Polygon'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9733,2024-07-23,2024-12-31,NaN,NaN,NaN,NaN,"{'coordinates': [[[37.703156729, 55.880530828], [37.7047011, 55.880597916], [37.704821983, 55.880024468], [37.703267964, 55.87993061], [37.703156729, 55.880530828]]], 'type': 'Polygon'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9734,2024-07-23,2024-12-31,NaN,NaN,NaN,NaN,"{'coordinates': [[[37.634549328, 55.818213136], [37.635359064, 55.818472086], [37.635752652, 55.818093949], [37.634952409, 55.817826954], [37.634549328, 55.818213136]]], 'type': 'Polygon'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9735,2024-04-23,2024-10-25,NaN,NaN,NaN,NaN,"{'coordinates': [[[37.635658777, 55.789345015], [37.635653417, 55.789568203]]], 'type': 'MultiLineString'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9736,2024-07-23,2025-05-30,NaN,NaN,NaN,NaN,"{'coordinates': [[[[37.835484338, 55.727888477], [37.835468658, 55.727859053], [37.835401114, 55.727870623], [37.835416794, 55.727900047], [37.835484338, 55.727888477]]], [[[37.836406939, 55.727396403], [37.836360433, 55.727370412], [37.836310823, 55.727397335], [37.836358521, 55.727423993], [37.836406939, 55.727396403]]], [[[37.83658804, 55.727480289], [37.836648348, 55.727455345], [37.836580419, 55.727422046], [37.836530837, 55.727452318], [37.83658804, 55.727480289]]], [[[37.836103279, 55.727550595], [37.836069885, 55.727531265], [37.836019087, 55.727558192], [37.836052482, 55.727577522], [37.836103279, 55.727550595]]], [[[37.835405017, 55.727772166], [37.835389332, 55.727742072], [37.83528742, 55.727759093], [37.83530428, 55.727787844], [37.835405017, 55.727772166]]]], 'type': 'MultiPolygon'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
for i in ecology.columns[2:23]:
    tmp = ecology[ecology[i].notna()][['start', 'end', i]]
    tmp['start'] = pd.to_datetime(tmp['start'], format='%Y-%m-%d')
    tmp['end'] = pd.to_datetime(tmp['end'], format='%Y-%m-%d')
    save_dataset('tmp')
    os.rename(SETS + 'tmp.gz', SETS + 'ecology_' + i + '.gz')

Сохранено 186 записей в tmp.gz
Сохранено 486 записей в tmp.gz
Сохранено 2216 записей в tmp.gz
Сохранено 113 записей в tmp.gz
Сохранено 6737 записей в tmp.gz
Сохранено 1449 записей в tmp.gz
Сохранено 19641 записей в tmp.gz
Сохранено 280 записей в tmp.gz
Сохранено 3662 записей в tmp.gz
Сохранено 37 записей в tmp.gz
Сохранено 108 записей в tmp.gz
Сохранено 2181 записей в tmp.gz
Сохранено 102488 записей в tmp.gz
Сохранено 24 записей в tmp.gz
Сохранено 26820 записей в tmp.gz
Сохранено 840 записей в tmp.gz
Сохранено 26 записей в tmp.gz
Сохранено 8 записей в tmp.gz
Сохранено 308 записей в tmp.gz
Сохранено 16 записей в tmp.gz
Сохранено 4089 записей в tmp.gz


In [7]:
exams['stupid'] = exams.apply(lambda x: x['PASSER_UNDER_160']/x['PASSES_OVER_220'] if x['PASSES_OVER_220'] != 0 else x['PASSER_UNDER_160']+(x['PASSER_UNDER_160']/10), axis=1)

In [8]:
exams_lite = exams[['global_id', 'start', 'end', 'stupid', 'geoData']]
exams_lite['start'] = pd.to_datetime(exams_lite['start'], format='%Y-%m-%d')
exams_lite['end'] = pd.to_datetime(exams_lite['end'], format='%Y-%m-%d')

In [9]:
save_dataset('exams_lite')

Сохранено 5542 записей в exams_lite.gz
